#Import Library

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
!pip install Sastrawi
!pip install Swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 8.2 MB/s 
     |████████████████████████████████| 281 kB 43.4 MB/s 
     |████████████████████████████████| 140 kB 58.9 MB/s 
  Created wheel for Swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=dc6be29407aee9359803bfa63313c8e896a491bc40be4bb7b6b7929de4624cf1
  Stored in directory: /root/.cache/pip/wheels/e8/1a/1b/46aa7b0fac16d3ce1bccaac14bfe4fad972642217ae78d8160
Successfully built Swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


#Load Data

In [ ]:
df = pd.read_csv('dataset_merged.csv')
df.head()

,id_akun,label,username,full_text
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...


#Data Cleaning + Tokenizing

In [ ]:
import string
import re 

def remove_emoji(text):
    text = re.sub(r'[^\x00-\x7f]', r'', str(text))
    text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', text)
    text = re.sub(r'\\u\w\w\w\w', '', text)
    return text

df['tweet_punc'] = df['full_text'].apply(lambda x: remove_emoji(x))
df.head(10)

,id_akun,label,username,full_text,tweet_punc
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...","@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,@SeaOfThieves Perbaruan terbaru?@Student080378...
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,@OGesports @TundraEsports_ @DreamHackDota Sial...
5,6,Neuroticism,babyreyfa,"“Kamu kuat, kamu hebat, kamu selalu bisa apa y...","Kamu kuat, kamu hebat, kamu selalu bisa apa ya..."
6,7,Agreeableness,zulfikarrafi,Tiba tiba di chat sama difollow ig dosen semes...,Tiba tiba di chat sama difollow ig dosen semes...
7,8,Neuroticism,baeseokjin12,@treasuremen Tdk bsa berkata-kata :)@SooHyunBi...,@treasuremen Tdk bsa berkata-kata :)@SooHyunBi...
8,9,Agreeableness,happy___hooman,@daeqleonhc semoga lu baca ini jugaaaa@convomf...,@daeqleonhc semoga lu baca ini jugaaaa@convomf...
9,10,Neuroticism,ShiroiRayn,Mending lanjut nonton apaa?@JAPANFESS Kenapa s...,Mending lanjut nonton apaa?@JAPANFESS Kenapa s...


In [ ]:
def mention(text):    
    return re.sub("@\S+", " ", text) #remove mention
df['tweet_punc'] = df['tweet_punc'].apply(mention)

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove retweet 
    text = re.sub("RT\s*", " ", text)
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['tweet_punc'] = df['tweet_punc'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['tweet_punc'] = df['tweet_punc'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
    
df['tweet_punc'] = df['tweet_punc'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['tweet_punc'] = df['tweet_punc'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['tweet_punc'] = df['tweet_punc'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['tweet_punc'] = df['tweet_punc'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tweet_tokens'] = df['tweet_punc'].apply(lambda x: word_tokenize_wrapper(x.lower()))

print('Tokenizing Result : \n') 

df.head(10)

Tokenizing Result : 



,id_akun,label,username,full_text,tweet_punc,tweet_tokens
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER Hyng seuri...,"[yaallah, lama, banget, ga, buka, twitter, hyn..."
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...,"[aku, kalau, kebangun, sebelum, solat, subuh, ..."
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...",Yuu masih kosong Yuu masih kosong Haiyuu Ajari...,"[yuu, masih, kosong, yuu, masih, kosong, haiyu..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,Perbaruan terbaru Bagaimana kabar motorku Mant...,"[perbaruan, terbaru, bagaimana, kabar, motorku..."
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,Sialan lupakan dan menangkan game selanjutnya ...,"[sialan, lupakan, dan, menangkan, game, selanj..."
5,6,Neuroticism,babyreyfa,"“Kamu kuat, kamu hebat, kamu selalu bisa apa y...",Kamu kuat kamu hebat kamu selalu bisa apa yang...,"[kamu, kuat, kamu, hebat, kamu, selalu, bisa, ..."
6,7,Agreeableness,zulfikarrafi,Tiba tiba di chat sama difollow ig dosen semes...,Tiba tiba di chat sama difollow ig dosen semes...,"[tiba, tiba, di, chat, sama, difollow, ig, dos..."
7,8,Neuroticism,baeseokjin12,@treasuremen Tdk bsa berkata-kata :)@SooHyunBi...,Tdk bsa berkatakata Udah pertanda direstui yuk...,"[tdk, bsa, berkatakata, udah, pertanda, direst..."
8,9,Agreeableness,happy___hooman,@daeqleonhc semoga lu baca ini jugaaaa@convomf...,semoga lu baca ini jugaaaa ngakak bgt pegawai ...,"[semoga, lu, baca, ini, jugaaaa, ngakak, bgt, ..."
9,10,Neuroticism,ShiroiRayn,Mending lanjut nonton apaa?@JAPANFESS Kenapa s...,Mending lanjut nonton apaa Kenapa sekarang pan...,"[mending, lanjut, nonton, apaa, kenapa, sekara..."


#Stopwords

In [ ]:
#get stopword indonesia
list_stopwords = stopwords.words('indonesian')
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['tweet_stopwords'] = df['tweet_tokens'].apply(lambda x: stopwords_removal(x))
df.head()

,id_akun,label,username,full_text,tweet_punc,tweet_tokens,tweet_stopwords
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER Hyng seuri...,"[yaallah, lama, banget, ga, buka, twitter, hyn...","[yaallah, banget, ga, buka, twitter, hyng, seu..."
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...,"[aku, kalau, kebangun, sebelum, solat, subuh, ...","[kebangun, solat, subuh, buka, webtoon, logika..."
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...",Yuu masih kosong Yuu masih kosong Haiyuu Ajari...,"[yuu, masih, kosong, yuu, masih, kosong, haiyu...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,Perbaruan terbaru Bagaimana kabar motorku Mant...,"[perbaruan, terbaru, bagaimana, kabar, motorku...","[perbaruan, terbaru, kabar, motorku, mantap, b..."
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,Sialan lupakan dan menangkan game selanjutnya ...,"[sialan, lupakan, dan, menangkan, game, selanj...","[sialan, lupakan, menangkan, game, dayummakhir..."


#Normalization

In [ ]:
normalized_word = pd.read_csv("dataset_merged.csv")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

df['tweet_normalized'] = df['tweet_stopwords'].apply(lambda x: normalized_term(x))

df.head(10)

,id_akun,label,username,full_text,tweet_punc,tweet_tokens,tweet_stopwords,tweet_normalized
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER Hyng seuri...,"[yaallah, lama, banget, ga, buka, twitter, hyn...","[yaallah, banget, ga, buka, twitter, hyng, seu...","[yaallah, banget, ga, buka, twitter, hyng, seu..."
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...,"[aku, kalau, kebangun, sebelum, solat, subuh, ...","[kebangun, solat, subuh, buka, webtoon, logika...","[kebangun, solat, subuh, buka, webtoon, logika..."
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...",Yuu masih kosong Yuu masih kosong Haiyuu Ajari...,"[yuu, masih, kosong, yuu, masih, kosong, haiyu...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,Perbaruan terbaru Bagaimana kabar motorku Mant...,"[perbaruan, terbaru, bagaimana, kabar, motorku...","[perbaruan, terbaru, kabar, motorku, mantap, b...","[perbaruan, terbaru, kabar, motorku, mantap, b..."
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,Sialan lupakan dan menangkan game selanjutnya ...,"[sialan, lupakan, dan, menangkan, game, selanj...","[sialan, lupakan, menangkan, game, dayummakhir...","[sialan, lupakan, menangkan, game, dayummakhir..."
5,6,Neuroticism,babyreyfa,"“Kamu kuat, kamu hebat, kamu selalu bisa apa y...",Kamu kuat kamu hebat kamu selalu bisa apa yang...,"[kamu, kuat, kamu, hebat, kamu, selalu, bisa, ...","[kuat, hebat, anak, papa, perempuan, terkuat, ...","[kuat, hebat, anak, papa, perempuan, terkuat, ..."
6,7,Agreeableness,zulfikarrafi,Tiba tiba di chat sama difollow ig dosen semes...,Tiba tiba di chat sama difollow ig dosen semes...,"[tiba, tiba, di, chat, sama, difollow, ig, dos...","[chat, difollow, ig, dosen, semester, gusti, i...","[chat, difollow, ig, dosen, semester, gusti, i..."
7,8,Neuroticism,baeseokjin12,@treasuremen Tdk bsa berkata-kata :)@SooHyunBi...,Tdk bsa berkatakata Udah pertanda direstui yuk...,"[tdk, bsa, berkatakata, udah, pertanda, direst...","[tdk, bsa, berkatakata, udah, pertanda, direst...","[tdk, bsa, berkatakata, udah, pertanda, direst..."
8,9,Agreeableness,happy___hooman,@daeqleonhc semoga lu baca ini jugaaaa@convomf...,semoga lu baca ini jugaaaa ngakak bgt pegawai ...,"[semoga, lu, baca, ini, jugaaaa, ngakak, bgt, ...","[semoga, lu, baca, jugaaaa, ngakak, bgt, pegaw...","[semoga, lu, baca, jugaaaa, ngakak, bgt, pegaw..."
9,10,Neuroticism,ShiroiRayn,Mending lanjut nonton apaa?@JAPANFESS Kenapa s...,Mending lanjut nonton apaa Kenapa sekarang pan...,"[mending, lanjut, nonton, apaa, kenapa, sekara...","[mending, nonton, apaa, panjat, tebingnya, sus...","[mending, nonton, apaa, panjat, tebingnya, sus..."


#Stemmer

In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tweet_stemmed'] = df['tweet_normalized'].swifter.apply(lambda x: get_stemmed_term(x))
df.head()

Streaming output truncated to the last 5000 lines.
haharidho : haharidho
pangestu : pangestu
adithio : adithio
risali : risali
kelasmu : kelas
panjaaaang : panjaaaang
begin : begin
againhuahaha : againhuahaha
lost : lost
contact : contact
putus : putus
heboh : heboh
it : it
makalah : makalah
han : han
id : id
line : line
kaliaaan : kaliaaan
grup : grup
kbr : kbr
hadir : hadir
mei : mei
heeeh : heeeh
solatnya : solatnya
youll : youll
see : see
top : top
world : world
krakatau : krakatau
steel : steel
ren : ren
aer : aer
mawar : mawar
kapitalisme : kapitalisme
hubungan : hubung
pertemanan : teman
do : do
anggaplah : anggap
ngutangin : ngutangin
inves : inves
hahahalelah : hahahalelah
tantesombong : tantesombong
kesedihan : sedih
pujian : puji
weakness : weakness
fiuh : fiuh
suaranya : suara
hahahasaat : hahahasaat
puncak : puncak
tertinggi : tinggi
jreng : jreng
hahatouchdown : hahatouchdown
jakarta : jakarta
mensyukuri : syukur
inidan : inidan
berdamai : damai
mumaka : mumaka
jack : jac

Pandas Apply:   0%|          | 0/33 [00:00<?, ?it/s]

,id_akun,label,username,full_text,tweet_punc,tweet_tokens,tweet_stopwords,tweet_normalized,tweet_stemmed
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER Hyng seuri...,"[yaallah, lama, banget, ga, buka, twitter, hyn...","[yaallah, banget, ga, buka, twitter, hyng, seu...","[yaallah, banget, ga, buka, twitter, hyng, seu...","[yaallah, banget, ga, buka, twitter, hyng, seu..."
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...,"[aku, kalau, kebangun, sebelum, solat, subuh, ...","[kebangun, solat, subuh, buka, webtoon, logika...","[kebangun, solat, subuh, buka, webtoon, logika...","[bangun, solat, subuh, buka, webtoon, logika, ..."
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...",Yuu masih kosong Yuu masih kosong Haiyuu Ajari...,"[yuu, masih, kosong, yuu, masih, kosong, haiyu...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde...","[yuu, kosong, yuu, kosong, haiyuu, ajarin, nde..."
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,Perbaruan terbaru Bagaimana kabar motorku Mant...,"[perbaruan, terbaru, bagaimana, kabar, motorku...","[perbaruan, terbaru, kabar, motorku, mantap, b...","[perbaruan, terbaru, kabar, motorku, mantap, b...","[baru, baru, kabar, motor, mantap, bacot, ni]"
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,Sialan lupakan dan menangkan game selanjutnya ...,"[sialan, lupakan, dan, menangkan, game, selanj...","[sialan, lupakan, menangkan, game, dayummakhir...","[sialan, lupakan, menangkan, game, dayummakhir...","[sial, lupa, menang, game, dayummakhirnya, per..."


In [ ]:
df.to_csv("Prepro.csv")

In [ ]:
import ast

df = pd.read_csv("Prepro.csv")

def join_text_list(texts):
  texts = ast.literal_eval(texts)
  return ' '.join([text for text in texts])

df["clear_tweet"] = df["tweet_stemmed"].apply(join_text_list)
df["clear_tweet"].head()

0    yaallah banget ga buka twitter hyng seuributuh...
1    bangun solat subuh buka webtoon logika gak jal...
2    yuu kosong yuu kosong haiyuu ajarin nder yuu k...
3                baru baru kabar motor mantap bacot ni
4    sial lupa menang game dayummakhirnya peringkat...
Name: clear_tweet, dtype: object

In [ ]:
df

,id_akun,label,username,full_text,tweet_punc,clear_tweet
0,1,Neuroticism,zzhllan,YAALLAH LAMA BANGET GA BUKA TWITTER @andiwhydi...,YAALLAH LAMA BANGET GA BUKA TWITTER Hyng seuri...,yaallah banget ga buka twitter hyng seuributuh...
1,2,Neuroticism,littlesheep_05,Aku kalau kebangun sebelum solat subuh malah b...,Aku kalau kebangun sebelum solat subuh malah b...,bangun solat subuh buka webtoon logika gak jal...
2,3,Extraversion,bayuajiii08,"@GAME_REFF https://t.co/KjLnz5lAro""@GAME_REFF ...",Yuu masih kosong Yuu masih kosong Haiyuu Ajari...,yuu kosong yuu kosong haiyuu ajarin nder yuu k...
3,4,Neuroticism,AgusNarestha,@SeaOfThieves Perbaruan terbaru?@Student080378...,Perbaruan terbaru Bagaimana kabar motorku Mant...,baru baru kabar motor mantap bacot ni
4,5,Neuroticism,TsuchiyaMirai,@OGesports @TundraEsports_ @DreamHackDota Sial...,Sialan lupakan dan menangkan game selanjutnya ...,sial lupa menang game dayummakhirnya peringkat...
5,6,Neuroticism,babyreyfa,"“Kamu kuat, kamu hebat, kamu selalu bisa apa y...",Kamu kuat kamu hebat kamu selalu bisa apa yang...,kuat hebat anak papa perempuan kuat cengeng ya...
6,7,Agreeableness,zulfikarrafi,Tiba tiba di chat sama difollow ig dosen semes...,Tiba tiba di chat sama difollow ig dosen semes...,chat difollow ig dosen semester gusti inget ke...
7,8,Neuroticism,baeseokjin12,@treasuremen Tdk bsa berkata-kata :)@SooHyunBi...,Tdk bsa berkatakata Udah pertanda direstui yuk...,tdk bsa berkatakata udah tanda restu yuk yuk j...
8,9,Agreeableness,happy___hooman,@daeqleonhc semoga lu baca ini jugaaaa@convomf...,semoga lu baca ini jugaaaa ngakak bgt pegawai ...,moga lu baca jugaaaa ngakak bgt pegawai start ...
9,10,Neuroticism,ShiroiRayn,Mending lanjut nonton apaa?@JAPANFESS Kenapa s...,Mending lanjut nonton apaa Kenapa sekarang pan...,mending nonton apaa panjat tebing susah banget...


In [ ]:
df = df.drop(['Unnamed: 0','tweet_tokens','tweet_stopwords','tweet_normalized','tweet_stemmed'], axis=1)
df.head(10)

KeyError: ignored

In [ ]:
df.to_csv("final_tweet.csv")